# Process nuclear data with NJOY

[NJOY](https://github.com/njoy/NJOY2016) is *the* reference code when it comes to process nuclear data.
Arguably, the primary goal of the code is to process tabulated nuclear data files into multiple data formats that are compatible with, among others, Monte Carlo particle transport codes such as [MCNP](https://mcnp.lanl.gov/) and [Serpent](http://montecarlo.vtt.fi/).

With a wrapper built around NJOY2016, SANDY simplifies the burden of [producing the NJOY input](https://raw.githubusercontent.com/njoy/NJOY2016-manual/master/njoy16.pdf) and running the code. Making use of some pre-defined NJOY parameters defined on expert-users' best-practices, SANDY can be consistently used to produce PENDF and ACE files of the nuclear data evaluations distributed by the major evaluated projects - e.g. JEFF, ENDF/B, JENDL - in the so-called [ENDF-6 format](https://www.nndc.bnl.gov/csewg/docs/endf-manual.pdf).

The PENDF and ACE formats are respectively:
 * PENDF: an ENDF-6 file with pointwise-reconstructed cross sectiions. It is a derived format where cross sections in MF section 3 are reconstructed from the resonance parameters in section MF=2 / MT=151. This cross sections can be further doppler-broadened at different temperatures. ALso, other derived cross sections can be added such as KERMA factors, dpa cross sections and gas production cross sections.
 * ACE: a compact ENDF file. A specific format used by MCNP and Serpent where all information available in a corresponding ENDF-6 file is stored.
 
> The conversion of an ENDF-6 file into ACE format requires first producing a PENDF file.

ENDF-6 files for neutron-induced data are generally distributed in individual evaluations per each single nuclide.
Each nuclear data project has its own evaluation of, say, uranium 235 (U-235) nuclear data, which might or not different from those produced by other projects.
Even more, each projects releases updates of their library more or less regularly.
For instance, the JEFF-3.3 nuclear data library was released in 2019 and followed its predecessor JEFF-3.2.

The distribution of these libraries is done by the project themselves, or it is centralized by organizations such as the OECD/NEA and the IAEA. Evaluated ENDF-6 files are generally availabel in public websites. The same cannot be said for the ACE files, that are generally problem dependent as different processing parameters can be used.

## Retrieve nuclear data file

To collect a nuclear data file one can call `.get_endf6_file` with three required arguments:
 * the identifier of the nuclear data library release (project and release number are separated by a `'_'`);
 * the nuclear data type, e.g. `'xs'` for neutron-induced data;
 * the nuclie ZAM identifier.
 
> A ZAM nuclide identifier is composed of three elements $Z$, $A$ and $M$, where:
> * $Z$ is the charge number
> * $A$ is the mass number
> * $M$ is the metastate level (0=ground, 1=1st level)
>
> The ZAM identifier is generated by combining the three elements following the rule
> $Z \times 10000 + A \times 10 + M$.
> Then,
> * hydrogen-1 (H-1) becomes 10010;
> * uranium-238 (U-238) becomes 922380;
> * americium-242 metastable (Am-242m) becomes 952421;
> * ...

To collect the hydrogen (H-1, ZAM=10010) file from JEFF-3.2 one can do the following.

In [1]:
import sandy

In [2]:
tape = sandy.get_endf6_file("jeff_32", "xs", 10010)
tape

'JEFF-3.2 Release - Neutron File March 2014                             0  0    0'


MAT  MF  MT 
125  1   451     1.001000+3 9.991673-1          0          0  ...
     2   151     1.001000+3 9.991673-1          0          0  ...
     3   1       1.001000+3 9.991673-1          0          0  ...
         2       1.001000+3 9.991673-1          0          0  ...
         102     1.001000+3 9.991673-1          0          0  ...
     4   2       1.001000+3 9.991673-1          0          1  ...
     6   102     1.001000+3 9.991673-1          0          2  ...
     33  1       1.001000+3 9.991673-1          0          0  ...
         2       1.001000+3 9.991673-1          0          0  ...
         102     1.001000+3 9.991673-1          0          0  ...
dtype: object

The ENDF-6 nuclear data evaluation is read by SANDY and imported in an instance of the `Endf6` class for further analysis. Still, to simply retrieve the file and save it to you working directory you can add `to_file=True`.

In [3]:
tape = sandy.get_endf6_file("jeff_32", "xs", 10010, to_file=True)

'JEFF-3.2 Release - Neutron File March 2014                             0  0    0'
INFO:  writing nuclear data to file '1-H-1.jeff_32'


We can do the same for different libraries.
For the moment SANDY can retrieve evaluations for neutron-induced nuclear data for:
 * JEFF-3.2 (use `jeff_32`)
 * JEFF-3.3 (use `jeff_33`)
 * JEFF-4.0T0 (use `jeff_40t0`)
 * ENDF/B-VII.1 (use `endfb_71`)
 
This example is for ENDF/B-VII.1,

In [4]:
tape = sandy.get_endf6_file("endfb_71", "xs", 10010, to_file=True)

INFO:  writing nuclear data to file '1-H-1.endfb_71'


## Processing with NJOY

### Get a PENDF file

`Endf6` method `get_pendf` runs NJOY to convert the ENDF-6 file evaluation into PENDF format.

In [5]:
pendf = tape.get_pendf()

INFO:  Zero or no temperature was requested, NJOY processing will stop after RECONR.
If you want to process 0K cross sections use `temperature=0.1`.



In [6]:
pendf

MAT  MF  MT 
125  1   451     1.001000+3 9.991673-1          2          0  ...
     2   151     1.001000+3 9.991673-1          0          0  ...
     3   1       1.001000+3 9.991673-1          0         99  ...
         2       1.001000+3 9.991673-1          0          0  ...
         102     1.001000+3 9.991673-1          0          0  ...
dtype: object

> If you work on a jupyter notebook the NJOY output is printed on the terminal!

If no temperature is specified the NJOY processing will stop after module RECONR (see [NJOY manual](https://raw.githubusercontent.com/njoy/NJOY2016-manual/master/njoy16.pdf)), that is, after reconstructing the cross sections but before any doppler broadening.

By specifying the temperature, e.g. `temperature=300` for 300 K, the processing will complete calling several NJOY modules according to an optimal processing sequence.
To see what NJOY modules are called and what options are used you can add option `verbose=True`.

In [7]:
pendf = tape.get_pendf(temperature=300, verbose=True, to_file=True)

moder
20 -21 /
reconr
-21 -22 /
'sandy runs njoy'/
125 0 0 /
0.001 0. /
0/
broadr
-21 -22 -23 /
125 1 0 0 0. /
0.001 /
300.0 /
0 /
thermr
0 -23 -24 /
0 125 20 1 1 0 0 1 221 0 /
300.0 /
0.001 10 /
heatr
-21 -24 -25 0 /
125 7 0 0 0 0 /
302 303 304 318 402 442 443 /
heatr
-21 -25 -26 0 /
125 4 0 0 0 0 /
444 445 446 447 /
gaspr
-21 -26 -27 /
purr
-21 -27 -28 /
125 1 1 20 32 0 /
300.0 /
1.00E+10 /
0 /
moder
-28 30 /
stop


The method output is again a `ENDF6` tape. Afterall, ENDF-6 and PENDF files are technically written according to the same format.
Notice however that NJOY removes some MF sections from the PENDF file.
In the example below MF4, MF33 and MF6/MT102 were removed.

Specific modules can be removed from the processing scheme by passing the module name as a kewyord argument and then adding `False`.
As an example, let's remove modules purr, thermr and heatr from the sequence! 

In [8]:
pendf = tape.get_pendf(temperature=300, purr=False, thermr=False, heatr=False, verbose=True, to_file=False)

moder
20 -21 /
reconr
-21 -22 /
'sandy runs njoy'/
125 0 0 /
0.001 0. /
0/
broadr
-21 -22 -23 /
125 1 0 0 0. /
0.001 /
300.0 /
0 /
gaspr
-21 -23 -24 /
moder
-24 30 /
stop


If no temperature was selected the sequence stops after module RECONR independently on the selected keyword arguments.

In [9]:
pendf = tape.get_pendf(purr=True, thermr=True, heatr=True, verbose=True, to_file=False)

INFO:  Zero or no temperature was requested, NJOY processing will stop after RECONR.
If you want to process 0K cross sections use `temperature=0.1`.



moder
20 -21 /
reconr
-21 -22 /
'sandy runs njoy'/
125 0 0 /
0.001 0. /
0/
moder
-22 30 /
stop


### Get an ACE file

An analogous method associated to any `Endf6` object is called `.get_ace` and runs the NJOY processing scheme even further up to the generation of a ACE file. The input syntax for this method is similar to that of `.get_pendf`, but this time the temperature is not an optional argument but it is required.

In [10]:
tape.get_ace(temperature=300, verbose=True)

moder
20 -21 /
reconr
-21 -22 /
'sandy runs njoy'/
125 0 0 /
0.001 0. /
0/
broadr
-21 -22 -23 /
125 1 0 0 0. /
0.001 /
300.0 /
0 /
thermr
0 -23 -24 /
0 125 20 1 1 0 0 1 221 0 /
300.0 /
0.001 10 /
heatr
-21 -24 -25 0 /
125 7 0 0 0 0 /
302 303 304 318 402 442 443 /
heatr
-21 -25 -26 0 /
125 4 0 0 0 0 /
444 445 446 447 /
gaspr
-21 -26 -27 /
purr
-21 -27 -28 /
125 1 1 20 32 0 /
300.0 /
1.00E+10 /
0 /
acer
-21 -28 0 50 70 /
1 0 1 .03 0 /
'sandy runs acer'/
125 300.0 /
1 1 /
/
stop


{'ace': '1001.03c', 'xsdir': '1001.03c.xsd'}

`.get_ace` takes the same keyword argument of `.get_pendf`.

However, the output is not anymore a `Endf6` instance, being a ACE file a different data format not handled directly by SANDY. The method output is a dictionary indicating the name of the generated ACE file and its corresponding XSDIR file (one line). The name of the file is the the ZA identifier according to the NNDC naming convention. The suffix (in this case `'03'`) is associated to the temperature (`'03'`=300 K) and defined by an internal routine. 